In [1]:
def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data

In [2]:
import os
# load in data
data_dir = './output (3).txt'
text = load_data(data_dir)

In [3]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 41173
Number of lines: 7594
Average number of words in each line: 67.34329734000526

The lines 0 to 10:
This is a story about a little girl who lives in an old house in the mountains Her name is Binchotan Each episode depicts a day in the life of Binchotan as she prepares her breakfast in the morning goes to the forest to gather vegetables does her household chores and rests at night after a days work She is surrounded by a group of close friends who add color to her ordinary yet simple life  
Kaitou Yumenosuke  is dream magician who loves eating the happy dreams of people 
In the future water has covered much of the Earth due to the effects of global warming The orphaned Maia Mizuki 15 just graduated from middle school and has already applied for employment in the elite paramilitary Ocean Agency part of the futuristic world government Only the best most intelligent and physically fit students are eligible for admission Maia the series 

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [4]:
from collections import Counter
import pickle

class Dataset():
    def __init__(self,text):
        self.SPECIAL_WORDS = {'PADDING': '<PAD>'}
        self.maxLen = 150
        self.dataset_path = './output (2).txt'
        
        (self.index_to_word, self.word_to_index) = (None,None)
        self.token_dict = self.token_lookup()
        
        
        self.int_text = self.preprocess_and_save_data(self.dataset_path)
        
    def create_lookup_tables(self, text):
        """
        Create lookup tables for vocabulary
        :param text: The text of tv scripts split into words
        :return: A tuple of dicts (vocab_to_int, int_to_vocab)
        """
        # TODO: Implement Function
        counter = Counter(text)
        vocab = sorted(counter, key = counter.get, reverse=True)

        vocab_to_int = {word: i for i, word in enumerate(vocab)}
        int_to_vocab = {i: word for word, i in vocab_to_int.items()}

        # return tuple
        return (vocab_to_int, int_to_vocab)
    
    def token_lookup(self):
        """
        Generate a dict to turn punctuation into a token.
        :return: Tokenized dictionary where the key is the punctuation and the value is the token
        """
        # TODO: Implement Function
        punc_dict = {'.': '||Period||',
            ',': '||Comma||',
            '"': '||Quotation_Mark||',
            ';': '||Semicolon||',
            '!': '||Exclamation_Mark||',
            '?': '||Question_Mark||',
            '(': '||Left_Parentheses||',
            ')': '||Right_Parentheses||',
            '-': '||Dash||',
            '\n': '||Return||'}
        return punc_dict
    
    def preprocess_and_save_data(self, dataset_path):
        """
        Preprocess Text Data
        """
        text = load_data(dataset_path)

        self.token_dict = self.token_lookup()
        
        for key, token in self.token_dict.items():
            text = text.replace(key, ' {} '.format(token))

        text = text.lower()
        text = text.split()
        
        (self.index_to_word, self.word_to_index) = self.create_lookup_tables(text + list(self.SPECIAL_WORDS.values()))

        int_text = [self.index_to_word[word] for word in text]
        pickle.dump((int_text, self.index_to_word, self.word_to_index, self.token_dict), open('preprocess.p', 'wb'))
        
        return int_text

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
dataset = Dataset(text)

In [6]:
def load_preprocess():
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    return pickle.load(open('preprocess.p', mode='rb'))

int_text, vocab_to_int, int_to_vocab, token_dict = load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

In [8]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    
    n_batches = len(words)//batch_size
    # only full batches
    batch_words = words[:n_batches*batch_size]
    
    # TODO: Implement function    
    features, targets = [], []
    
    L = len(batch_words)
    for index in range(L - sequence_length):
        features.append(batch_words[index : index + sequence_length])
        targets.append(batch_words[index + sequence_length])   
        

    data = TensorDataset(torch.from_numpy(np.asarray(features)), torch.from_numpy(np.asarray(targets)))
    data_loader = torch.utils.data.DataLoader(data, shuffle=False, batch_size=batch_size)

    # return a dataloader
    return data_loader


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [9]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # TODO: Implement function
        
        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define model layers
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        
        # drpout layer
        self.dropout = nn.Dropout(dropout)        

        # linear layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function  
        batch_size = nn_input.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        output = self.dropout(lstm_out)
        output = self.fc(output)
        
        # reshape to be batch_size first
        output = output.view(batch_size, -1, self.output_size)
        out = output[:, -1] # get last batch of labels       
        # return one batch of output word scores and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        weight = next(self.parameters()).data
        
        # initialize hidden state with zero weights, and move to GPU if available
            
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
 
        
        return hidden


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [10]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # move data to GPU, if available
    
    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()
    
    # perform backpropagation and optimization
    hidden = tuple([each.data for each in hidden])
    rnn.zero_grad()
    output, hidden = rnn(inp, hidden)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()


    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [12]:
# Data params
# Sequence Length
sequence_length = 8  # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [13]:
# Training parameters
# Number of Epochs
num_epochs = 20
# Learning Rate
learning_rate = 0.001

# Model parameters
L = len(vocab_to_int)
# Vocab size
vocab_size = L
# Output size
output_size = L
# Embedding Dimension
embedding_dim = 200
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 100

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [14]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# from workspace_utils import active_session

# with active_session():

def save_model(filename, decoder):
    save_filename = "trained_rnn" + '.pt'
    torch.save(decoder, save_filename)

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.3)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
save_model('trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 20 epoch(s)...
Epoch:    1/20    Loss: 7.613448543548584

Epoch:    1/20    Loss: 7.200689520835876

Epoch:    1/20    Loss: 7.177126779556274

Epoch:    1/20    Loss: 7.076960906982422

Epoch:    2/20    Loss: 6.715315039795224

Epoch:    2/20    Loss: 6.708220777511596

Epoch:    2/20    Loss: 6.671909074783326

Epoch:    2/20    Loss: 6.6453996181488035

Epoch:    3/20    Loss: 6.434490010289863

Epoch:    3/20    Loss: 6.380907297134399

Epoch:    3/20    Loss: 6.346472873687744

Epoch:    3/20    Loss: 6.298454165458679

Epoch:    4/20    Loss: 6.156661482140569

Epoch:    4/20    Loss: 6.0715707635879514

Epoch:    4/20    Loss: 6.025762119293213

Epoch:    4/20    Loss: 5.969320387840271

Epoch:    5/20    Loss: 5.87946887063508

Epoch:    5/20    Loss: 5.779364461898804

Epoch:    5/20    Loss: 5.699770283699036

Epoch:    5/20    Loss: 5.648444051742554

Epoch:    6/20    Loss: 5.564130249589977

Epoch:    6/20    Loss: 5.467204918861389

Epoch:    6/20    Loss: 5

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [16]:
import torch

def load_model(filename):
    save_filename = "trained_rnn" + '.pt'
    return torch.load(save_filename)

_, vocab_to_int, int_to_vocab, token_dict = load_preprocess()
trained_rnn = load_model('trained_rnn')

In [62]:
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, len(prime_id)), pad_value)
    current_seq = np.asarray([prime_id])
    predicted = [int_to_vocab[i] for i in prime_id]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
#         print(current_seq)
        current_seq = np.roll(current_seq.cpu(), -1, 1)
        
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n ------------------------------ \n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

In [63]:
vocab_to_int["care"]

771

In [69]:
import pandas as pd
from random import randint

df = pd.read_csv("outputfile.csv")
value = randint(0, len(vocab_to_int))

data_dir = './output (3).txt'
text = load_data(data_dir)

prime_word = text[value:value+150]
gen_length = 400 # modify the length to your preference

sequence = []

for i in prime_word.split(" "):
    try:
        sequence.append(vocab_to_int[i.lower()])
    except:
        continue

        
print('----- Generating with seed: "' + prime_word + '" -----')
print()
print()
pad_word = dataset.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, sequence, int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

----- Generating with seed: "demy realize that there are no clubs where they fit in they band together to form the Student Cultural Society or StuCS for short The club consists of" -----


realize that there are no clubs where they fit in they band together to form the student society or for short the club consists of a newly boss is also a long world who is a tight knit girl who lives his way for her own yui falls yuzu to help her only to her happy her father but she also be many people from the future but they discover the fruit of her life falls into an undercover coup a empty socialite tosses his allergy that would be a normal mysterious world source ann
 ------------------------------ 
a music video for the song agency releases
 ------------------------------ 
an small music video by denki grooves song de oni
 ------------------------------ 
miko mido has been obsessed with two lessons and bite
 ------------------------------ 
the story concerns gauche a fondness to determine the 

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [61]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()